In [1]:
using Distributions
using Base.Test
using StatsBase
cd("../Source/")
include("types.jl")
include("functions_neighbors.jl")
include("functions_types.jl")
include("functions_symmetric_energy.jl")
include("functions_mc.jl")
include("functions_thermalization.jl")
include("functions_observables.jl")
cd("../Notebooks/")
using Plots
pyplot()

Plots.PyPlotBackend()

In [2]:
THERM_FRAC = 1/10
DT_MAX = 10000

@everywhere const two_pi = 2π

# We run a simulation with the parameters
g = 0.3    # Gauge coupling
ν = 0.3    # Anisotropy

# Other parameters
L = 64# System length
L₃ = 10
T = 0.12 #0.11039   # Temperature
γ = 1.0    # Order parameter amplitud
κ₅ = 1.0
f = 1.0/L
println("f set to $(f)")
β = 1/T
println("β set to $(β)")

# Create system
syst = SystConstants(L, L₃, γ, 1/g^2, ν, κ₅, f, β)
sim = Controls(π/3, 0.4, 3.0)

# Measurement parameters
M = 300    # Number of measurements
Δt = 400  # Interval between measurements (MCS)
# Construct k-matrix where the horizontal axis contains kx ∈ [-π, π), while
# the vertical axis contain ky ∈ [-π, π) at the second component
k_matrix = [[2π/L*(x-1-L/2), 2π/L*(L/2-y)] for y=1:L, x=1:L];

f set to 0.015625
β set to 8.333333333333334


In [3]:
ψ = State(2, syst);

In [4]:
M = 8
ψ_list = [ψ for i = 1:M];

In [5]:
# -------------------------------------------------------------------------------------------------
# Saves the state to file with specified filename. If mode is set to "a" then the state is appended
# to the file.
function save(ψ::State, filename::AbstractString, mode::AbstractString="w")
    if mode != "w" && mode != "a"
        println("$(mode) is not a valid file-opening option.")
        return 0
    end
    println("Saving state to file $(filename)")
    L = ψ.consts.L
    L₃ = ψ.consts.L₃
    open(filename, mode) do f
        # Write line indicating start of state
        write(f, "state start\n")
        # Writing system constants on top of the file
        write(f, "$(ψ.consts.L)\n")
        write(f, "$(ψ.consts.L₃)\n")
        write(f, "$(ψ.consts.γ)\n")
        write(f, "$(ψ.consts.g⁻²)\n")
        write(f, "$(ψ.consts.ν)\n")
        write(f, "$(ψ.consts.κ₅)\n")
        write(f, "$(ψ.consts.f)\n")
        write(f, "$(ψ.consts.β)\n")
        # Then starting writing the state lattice
        for z=1:L₃, x=1:L, y=1:L
            ϕ = ψ.lattice[y,x,z]
            write(f, "$(ϕ.A[1]):$(ϕ.A[2]):$(ϕ.A[3]):$(ϕ.θ⁺):$(ϕ.θ⁻):$(ϕ.u⁺):$(ϕ.u⁻)\n")
        end
        write(f, "end\n")
    end
    return 1
end



save (generic function with 2 methods)

In [6]:
function save(ψ_list::Array{State, 1}, filename::AbstractString="state_list.data")
    open(filename, "w") do f
        write(f, "state array\n")
        write(f, "$(length(ψ_list))\n")
        write(f, "$(ψ_list[1].consts.L):$(ψ_list[1].consts.L₃):$(ψ_list[1].consts.γ):$(ψ_list[1].consts.g⁻²):")
        write(f, "$(ψ_list[1].consts.ν):$(ψ_list[1].consts.κ₅):$(ψ_list[1].consts.f):$(ψ_list[1].consts.β)\n")
        for ψ in ψ_list
            write(f, "state start\n")
            for z=1:L₃, h=1:L, v=1:L
                ϕ = ψ.lattice[v,h,z]
                write(f, "$(ϕ.A[1]):$(ϕ.A[2]):$(ϕ.A[3]):$(ϕ.θ⁺):$(ϕ.θ⁻):$(ϕ.u⁺):$(ϕ.u⁻)\n")
            end
        end
    end
    return 1
end

save (generic function with 4 methods)

In [9]:
function addToList(ψ::State, filename::AbstractString)
    M₀ = 0
    # Get length of existing list and check that constants are the same as state.
    open(filename, "r") do file
        line = readline(file)
        if line != "state array"
            println("ERROR: Start of file is $(line)")
            throw(DomainError())
        end
        M₀ = parse(Int64, readline(file))
        M₀ > 0 || throw(DomainError())
        c_values = split(readline(file), ":")
        L = parse(Int64, c_values[1])
        L > 0 || throw(DomainError())
        L₃ = parse(Int64, c_values[2])
        L₃ > 0 || throw(DomainError())
        γ = parse(Float64, c_values[3])
        g⁻² = parse(Float64, c_values[4])
        ν = parse(Float64, c_values[5])
        κ₅ = parse(Float64, c_values[6])
        f = parse(Float64, c_values[7])
        β = parse(Float64, c_values[8])
        if !(ψ.consts.L == L && ψ.consts.L₃ == L₃ && ψ.consts.γ == γ && ψ.consts.g⁻² == g⁻² && ψ.consts.ν == ν
                && ψ.consts.κ₅ == κ₅ && ψ.consts.f == f && ψ.consts.β == β)
            throw(DomainError)
        end
    end
    # Save lattice to new state at the end
    open(filename, "a") do file
        write(file, "state start\n")
        for z=1:L₃, h=1:L, v=1:L
            ϕ = ψ.lattice[v,h,z]
            write(file, "$(ϕ.A[1]):$(ϕ.A[2]):$(ϕ.A[3]):$(ϕ.θ⁺):$(ϕ.θ⁻):$(ϕ.u⁺):$(ϕ.u⁻)\n")
        end
    end
    # Update length of list by 1.
    open(filename, "r+") do file
        line = readline(file)
        write(file, "$(M₀+1)\n")
    end
    return 1
end

addToList (generic function with 1 method)

In [10]:
save(ψ_list, "state_list.data")
ψ₂ = State(2, syst)
addToList(ψ₂, "state_list.data")

1

In [11]:
# load
# -------------------------------------------------------------------------------------------------
# The converse of the save function above: takes a text file where a state is saved and converts
# it into a State object which it return.
function readState(filename::AbstractString)
    open(filename, "r") do file
        line = readline(file)
        if line != "state start"
            throw(DomainError())
        end
        L = parse(Int64, readline(file))
        L₃ = parse(Int64, readline(file))
        γ = parse(Float64, readline(file))
        g⁻² = parse(Float64, readline(file))
        ν = parse(Float64, readline(file))
        κ₅ = parse(Float64, readline(file))
        f = parse(Float64, readline(file))
        β = parse(Float64, readline(file))
        syst = SystConstants(L, L₃, γ, g⁻², ν, κ₅, f, β)
        
        lattice = Array{LatticeSite, 3}(L,L,L₃)
        for z_pos = 1:L₃, h_pos = 1:L, v_pos = 1:L
            ϕ_values = split(readline(file), ":")
            A₁ = parse(Float64, ϕ_values[1])
            A₂ = parse(Float64, ϕ_values[2])
            A₃ = parse(Float64, ϕ_values[3])
            θ⁺ = parse(Float64, ϕ_values[4])
            θ⁻ = parse(Float64, ϕ_values[5])
            u⁺ = parse(Float64, ϕ_values[6])
            u⁻ = parse(Float64, ϕ_values[7])
            lattice[v_pos,h_pos,z_pos] = LatticeSite([A₁, A₂, A₃], θ⁺, θ⁻, u⁺, u⁻)
        end
        
        nbl = latticeNeighbors(lattice,L,L₃)
        nnbl = latticeNextNeighbors(lattice,L,L₃)
        nnnbl = latticeNNNeighbors(lattice,L,L₃)
        
        return State(lattice, syst, nbl, nnbl, nnnbl)
    end
end


readState (generic function with 1 method)

In [12]:
function loadStates(filename::AbstractString)
    open(filename, "r") do file
        line = readline(file)
        if line != "state array"
            throw(DomainError())
        end
        M = parse(Int64, readline(file))
        M > 0 || throw(DomainError())
        ψ_l = Array{State, 1}(M)
        c_values = split(readline(file), ":")
        L = parse(Int64, c_values[1])
        L > 0 || throw(DomainError())
        L₃ = parse(Int64, c_values[2])
        L₃ > 0 || throw(DomainError())
        γ = parse(Float64, c_values[3])
        g⁻² = parse(Float64, c_values[4])
        ν = parse(Float64, c_values[5])
        κ₅ = parse(Float64, c_values[6])
        f = parse(Float64, c_values[7])
        β = parse(Float64, c_values[8])
        for i = 1:M
            line = readline(file)
            if line != "state start"
                throw(DomainError())
            end
            syst = SystConstants(L,L₃,γ,g⁻²,ν,κ₅,f,β)
            lattice = Array{LatticeSite, 3}(L,L,L₃)
            for z_pos = 1:L₃, h_pos = 1:L, v_pos = 1:L
                ϕ_values = split(readline(file), ":")
                A₁ = parse(Float64, ϕ_values[1])
                A₂ = parse(Float64, ϕ_values[2])
                A₃ = parse(Float64, ϕ_values[3])
                θ⁺ = parse(Float64, ϕ_values[4])
                θ⁻ = parse(Float64, ϕ_values[5])
                u⁺ = parse(Float64, ϕ_values[6])
                u⁻ = parse(Float64, ϕ_values[7])
                lattice[v_pos,h_pos,z_pos] = LatticeSite([A₁, A₂, A₃], θ⁺, θ⁻, u⁺, u⁻)
            end

            nbl = latticeNeighbors(lattice,L,L₃)
            nnbl = latticeNextNeighbors(lattice,L,L₃)
            nnnbl = latticeNNNeighbors(lattice,L,L₃)

            ψ_l[i] = State(lattice, syst, nbl, nnbl, nnnbl)
        end
        return ψ_l
    end
end

loadStates (generic function with 1 method)

In [29]:
ψ_l = loadStates("state_list.data");

In [14]:
ψ_ll = [ψ_list..., ψ₂];

In [30]:
ψ_l[1].lattice[1,1,1].A

3-element Array{Float64,1}:
  25.0831 
  -6.74244
 -12.5871 

In [32]:
length(ψ_l) == length(ψ_ll)
for i = 1:length(ψ_l)
    @test ψ_l[i] == ψ_ll[i]
end

In [40]:
typeof(ψ_l[1])

State